In [6]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.33 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_7_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [10]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [11]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20,30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=149468, Thu Apr 16 18:35:56 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.174108761262080e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 2.031080270269017e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22611.82334804] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [20823.47463217] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.866385131235248e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   28    532 7.424998634963351e+03 1.1e+00 2.10e+00  2e+00  2e+00 0:03.3
   63   1197 3.516199044591169e+03 1.2e+00 1.89e+00  2e+00  2e+00 0:07.4
  100   1900 -1.504363170647670e+03 1.2e+00 1.40e+00  1e+00  1e+00 0:11.8
  152   2888 -1.012344274414434e+04 1.2e+00 8.82e-01  9e-01  9e-01 0:17.8
  200   3800 -1.697477668389479e+04 1.2e+00 5.30e-01  5e-01  5e-01 0:23.5
  268   5092 -2.026891893509584e+04 1.2e+00 2.62e-01  3e-01  3e-01 0:31.5
  300   5700 -2.097223563991850e+04 1.2e+00 1.82e-01  2e-01  2e-01 0:35.2
  385   7315 -2.159663522889238e+04 1.2e+00 7.45e-02  7e-02  8e-02 0:45.3
  400   7600 -2.161000256519732e+04 1.2e+00 6.42e-02  6e-02  6e-02 0:47.1
  500   9500 -2.169842521241849e+04 1.2e+00 2.04e-02  2e-02  2e-02 0:58.7
  600  11400 -2.170577503312411e+04 1.2e+00 6.79e-03  7e-03  7e-03 1:10.5
  700  13300 -2.170682664560686e+04 1.3e+00 2.71e-03  3e-03  3e-03 1:22.4
  800  15200 -2.170695306108230e+04 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [23153.52062679] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23153.520627, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23153.520627, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Thu Apr 16 18:58:10 2020)
final/bestever f-value = 2.315352e+04 2.315352e+04
incumbent solution: [ 4.08831998 -2.73819833  0.25941661  3.93241953 -4.96344155  1.40625653
  4.98029507  2.48728932 ...]
std deviations: [2.43760188 2.43778705 2.4375641  2.43784386 2.4376631  2.43771368
 2.43723598 2.43792541 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=106288, Thu Apr 16 18:58:10 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.161870138750285e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 2.029737956835255e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

10200 193800 -3.425343759824973e+04 6.3e+00 2.41e-09  2e-09  8e-09 19:18.7


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-34253.437598, sigma=2.30e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=10228)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-34253.437598, sigma=2.41e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=10284)
  ')')


10300 195700 -3.425343759824973e+04 6.3e+00 2.40e-09  2e-09  8e-09 19:30.1
10400 197600 -3.425343759824973e+04 6.3e+00 2.47e-09  2e-09  8e-09 19:41.4


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-34253.437598, sigma=2.41e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=10430)
  ')')


10500 199500 -3.425343759824973e+04 6.3e+00 2.50e-09  2e-09  9e-09 19:52.6
10527 200013 -3.425343759824973e+04 6.3e+00 2.57e-09  2e-09  9e-09 19:55.7
termination on maxfevals=200000 (Fri Apr 17 01:46:02 2020)
final/bestever f-value = -3.425344e+04 -3.425344e+04
incumbent solution: [-0.05018879  5.          5.         -5.         -5.         -5.
  0.81070737 -5.         ...]
std deviations: [2.87524413e-09 2.44398113e-09 1.94116041e-09 2.27023719e-09
 2.12083571e-09 2.41543455e-09 2.84268013e-09 2.24651471e-09 ...]


In [12]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F7_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:200])
    SVM_Fun [i] = F7(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F7(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4151.53796328]), count=array([10])) 0.0
SVM
ModeResult(mode=array([22182.87223956]), count=array([1])) 3258.185740946945
ELN
ModeResult(mode=array([23739.72898403]), count=array([1])) 1275.3766991857328
